In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window
import pandas as pd
import os
spark = SparkSession.builder.appName('bigData').getOrCreate()

In [3]:
path = '/mnt/c/Users/wacke/Documents/Master/3. Semester/Big Data Analytics/Assignment2'

In [4]:
raw_df = spark.read.text(os.path.join(path, 'pfsense.log'))
print(raw_df.printSchema())
print(type(raw_df))
len_df = raw_df.count()
print('Shape df:')
print((len_df, len(raw_df.columns)))

root
 |-- value: string (nullable = true)

None
<class 'pyspark.sql.dataframe.DataFrame'>
Shape df:
(15441191, 1)


In [5]:
raw_df.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                        |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Sep  7 11:45:47 monitoring systemd[1]: Stopping System Logging Service...                                                                                                                                                                    |
|Sep  7 11:45:47 monitoring rsyslogd:  [

In [6]:
def is_int(element):
    try:
        float(element)
        return True
    except ValueError:
        return False
    
def get_csvPart(item):
    return item['value'].split(': ')[1]
    
def get_ruleNumber(item):
    return get_csvPart(item).split(',')[0]

def get_ipVersion(item):
    return get_csvPart(item).split(',')[8]

In [9]:
csvPart = raw_df.rdd.map(lambda row: get_csvPart(row).split(','))

In [11]:
csvPart.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 3.0 failed 1 times, most recent failure: Lost task 2.0 in stage 3.0 (TID 25, 192.168.224.139, executor driver): java.lang.OutOfMemoryError: Java heap space

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2059)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2008)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2007)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2007)
	at org.apache.spark.scheduler.DAGSchedule

In [ ]:
#csvPart = [get_csvPart(item).split(',') for item in raw_df.take(1000000) if is_int(get_ruleNumber(item)) and get_ipVersion(item)=='4']
csvPart = [get_csvPart(item).split(',') for item in raw_df.collect() if is_int(get_ruleNumber(item)) and get_ipVersion(item)=='4']

In [ ]:
def add_missing_col(array):
    if len(array) < 30:
        missing_cols = 30-len(array)
        for i in range(missing_cols):
            array.insert(len(array), '')
    return array

In [ ]:
for array in csvPart:
    array = add_missing_col(array)

In [ ]:
'''
schema = StructType([
    StructField('rule_number', IntegerType(), True),
    StructField('sub_rule_nuber', IntegerType(), True),
    StructField('anchor', IntegerType(), True),
    StructField('tracker_id', StringType(), True),
    StructField('real_interface', StringType(), True),
    StructField('reason', StringType(), True),
    StructField('action', StringType(), True),
    StructField('real_interface', StringType(), True),
    StructField('traffic_direction', StringType(), True),
    StructField('ip_version', IntegerType(), True),
    StructField('tos', StringType(), True),
    StructField('ecn', StringType(), True),
    StructField('ttl', IntegerType(), True),
    StructField('ipv4_id', IntegerType(), True),
    StructField('offset', IntegerType(), True),
    StructField('ipv4_id', IntegerType(), True),
    StructField('flags', StringType(), True),
    StructField('protocoll_id', IntegerType(), True),
    StructField('protocoll_text', StringType(), True),
    StructField('length', IntegerType(), True),
    StructField('length', IntegerType(), True),
    StructField('source_ip', StringType(), True),
    StructField('destination_ip', StringType(), True),
    StructField('source_port', IntegerType(), True),
    StructField('destination_port', IntegerType(), True),
    StructField('data_length', StringType(), True),
    StructField('tcp_flag', StringType(), True),
    StructField('sequence_num', StringType(), True),
    StructField('window', IntegerType(), True),
    StructField('urg', IntegerType(), True),
    StructField('options', StringType(), True),
])
'''
schema_csvPart = StructType([
    StructField('rule_number', StringType(), True),
    StructField('sub_rule_nuber', StringType(), True),
    StructField('anchor', StringType(), True),
    StructField('tracker_id', StringType(), True),
    StructField('real_interface', StringType(), True),
    StructField('reason', StringType(), True),
    StructField('action', StringType(), True),
    StructField('traffic_direction', StringType(), True),
    StructField('ip_version', StringType(), True),
    StructField('tos', StringType(), True),
    StructField('ecn', StringType(), True),
    StructField('ttl', StringType(), True),
    StructField('ipv4_id', StringType(), True),
    StructField('offset', StringType(), True),
    StructField('flags', StringType(), True),
    StructField('protocoll_id', StringType(), True),
    StructField('protocoll_text', StringType(), True),
    StructField('length', StringType(), True),
    StructField('source_ip', StringType(), True),
    StructField('destination_ip', StringType(), True),
    StructField('source_port', StringType(), True),
    StructField('destination_port', StringType(), True),
    StructField('data_length', StringType(), True),
    StructField('tcp_flag', StringType(), True),
    StructField('sequence_num', StringType(), True),
    StructField('ack', StringType(), True),
    StructField('window', StringType(), True),
    StructField('urg', StringType(), True),
    StructField('options', StringType(), True),
    StructField('extra', StringType(), True)
])

In [ ]:
# Convert list to RDD
rdd_csvPart = spark.sparkContext.parallelize(csvPart)

# Create data frame
df_csvPart = spark.createDataFrame(rdd_csvPart,schema_csvPart)
del rdd_csvPart

In [ ]:
def get_timestamp(item):
    return item['value'].split(': ')[0].split(' gw01')[0]

In [ ]:
timestamp = [get_timestamp(item) for item in raw_df.take(1000000) if is_int(get_ruleNumber(item))]

In [ ]:
del raw_df

In [ ]:
schema_timestamp = StructType([
    StructField('timestamp', StringType(), True)
])

In [ ]:
df_timestamp = spark.createDataFrame(timestamp, StringType())

In [ ]:
df_csvPart = df_csvPart.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))
df_timestamp = df_timestamp.withColumn('row_index', row_number().over(Window.orderBy(monotonically_increasing_id())))

In [ ]:
df_v4 = df_timestamp.join(df_csvPart, on=["row_index"]).drop("row_index")
del df_timestamp
del df_csvPart

In [ ]:
df_v4 = df_v4.withColumnRenamed('value', 'timestamp')
df_v4.show()

In [ ]:
#df_v4.write.csv(os.path.join(path, 'df_v4.log'))

In [ ]:
df_pandas = df_v4.toPandas()

In [ ]:
df_pandas